Import Libraries

In [2]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb

Load and Display Data

In [3]:
df = pd.read_csv('Dataset/StrongPassword.csv')
df.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,...,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,...,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,...,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,...,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,...,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818


Feature Extraction Functions

In [4]:
dwell_columns = [col for col in df.columns if col.startswith('H.')]
latency_columns = [col for col in df.columns if col.startswith('DD.')]
flight_columns = [col for col in df.columns if col.startswith('UD.')]

def extract_first_order_features(df):
    # Select relevant columns for first-order features
    dwell_times_df = df[dwell_columns]
    latency_df = df[latency_columns]
    flight_times_df = df[flight_columns]

    return pd.concat([dwell_times_df, latency_df, flight_times_df], axis=1)

def extract_second_order_features(first_order_df):
    # Calculate mean and standard deviation for each first-order feature type
    second_order_features = {
        'mean_dwell_time': first_order_df[dwell_columns].mean(axis=1),
        'std_dwell_time': first_order_df[dwell_columns].std(axis=1),
        'mean_latency': first_order_df[latency_columns].mean(axis=1),
        'std_latency': first_order_df[latency_columns].std(axis=1),
        'mean_flight_time': first_order_df[flight_columns].mean(axis=1),
        'std_flight_time': first_order_df[flight_columns].std(axis=1),
    }
    return pd.DataFrame(second_order_features)

Extract Features

In [5]:
first_order_df = extract_first_order_features(df)
second_order_df = extract_second_order_features(first_order_df)
combined_df = pd.concat([first_order_df, second_order_df], axis=1)

In [6]:
first_order_df

,H.period,H.t,H.i,H.e,H.five,H.Shift.r,H.o,H.a,H.n,H.l,...,UD.period.t,UD.t.i,UD.i.e,UD.e.five,UD.five.Shift.r,UD.Shift.r.o,UD.o.a,UD.a.n,UD.n.l,UD.l.Return
0,0.1491,0.1069,0.1169,0.1417,0.1146,0.1067,0.1016,0.1349,0.0932,0.1338,...,0.2488,0.0605,0.1043,1.0468,1.4909,0.6523,0.1120,0.0135,0.2583,0.2171
1,0.1111,0.0694,0.0908,0.0829,0.0689,0.1570,0.1066,0.1412,0.1146,0.0839,...,0.2340,0.0589,0.0449,1.1141,0.7133,0.6307,0.0618,0.1146,0.1496,0.1917
2,0.1328,0.0731,0.0821,0.0808,0.0892,0.1454,0.1365,0.1621,0.1172,0.1085,...,0.0744,0.0560,0.0721,0.9600,0.5311,0.5741,0.1566,0.0711,0.1533,0.1762
3,0.1291,0.1059,0.1040,0.0900,0.0913,0.1454,0.0956,0.1457,0.0866,0.0845,...,0.1224,0.1436,0.0998,0.9656,1.1651,0.6096,0.0574,0.0172,0.1475,0.2387
4,0.1249,0.0895,0.0903,0.0805,0.0742,0.1243,0.0430,0.1312,0.0884,0.0903,...,0.1068,0.0781,0.0686,0.7824,0.8213,0.6389,0.1545,0.0270,0.1633,0.1614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20395,0.0884,0.1095,0.0945,0.1328,0.1000,0.0863,0.0944,0.1219,0.0820,0.1005,...,-0.0199,0.0195,-0.0188,-0.0502,0.1398,0.1285,0.1122,0.0164,0.0509,0.1049
20396,0.0655,0.0910,0.0916,0.1256,0.0987,0.0892,0.0776,0.1008,0.1037,0.1445,...,-0.0025,0.0238,-0.0280,-0.0404,0.1454,0.0317,0.0201,-0.0496,-0.0169,0.0761
20397,0.0939,0.1008,0.0721,0.0903,0.0733,0.0742,0.0599,0.0913,0.0689,0.1034,...,0.0250,0.0114,-0.0259,0.1142,0.1457,0.0958,0.0505,0.0256,0.0622,0.0983
20398,0.0923,0.0913,0.0992,0.1016,0.0820,0.0729,0.0728,0.0882,0.0576,0.0979,...,0.0371,0.0077,-0.0095,-0.0446,0.2061,0.0873,0.0383,-0.0061,0.0121,0.0938


In [7]:
second_order_df

,mean_dwell_time,std_dwell_time,mean_latency,std_latency,mean_flight_time,std_flight_time
0,0.115782,0.022530,0.54039,0.493923,0.42045,0.492082
1,0.100100,0.029165,0.43400,0.361336,0.33136,0.363031
2,0.111109,0.029826,0.39526,0.298385,0.28249,0.304275
3,0.105400,0.023856,0.46450,0.405723,0.35669,0.410564
4,0.092582,0.025764,0.39389,0.311851,0.30023,0.315000
...,...,...,...,...,...,...
20395,0.101364,0.015347,0.14936,0.063340,0.04833,0.068961
20396,0.100727,0.022356,0.11479,0.066040,0.01597,0.059185
20397,0.083509,0.014318,0.14309,0.054281,0.06028,0.052993
20398,0.086264,0.013601,0.12780,0.069816,0.04222,0.071676


In [8]:
combined_df

,H.period,H.t,H.i,H.e,H.five,H.Shift.r,H.o,H.a,H.n,H.l,...,UD.o.a,UD.a.n,UD.n.l,UD.l.Return,mean_dwell_time,std_dwell_time,mean_latency,std_latency,mean_flight_time,std_flight_time
0,0.1491,0.1069,0.1169,0.1417,0.1146,0.1067,0.1016,0.1349,0.0932,0.1338,...,0.1120,0.0135,0.2583,0.2171,0.115782,0.022530,0.54039,0.493923,0.42045,0.492082
1,0.1111,0.0694,0.0908,0.0829,0.0689,0.1570,0.1066,0.1412,0.1146,0.0839,...,0.0618,0.1146,0.1496,0.1917,0.100100,0.029165,0.43400,0.361336,0.33136,0.363031
2,0.1328,0.0731,0.0821,0.0808,0.0892,0.1454,0.1365,0.1621,0.1172,0.1085,...,0.1566,0.0711,0.1533,0.1762,0.111109,0.029826,0.39526,0.298385,0.28249,0.304275
3,0.1291,0.1059,0.1040,0.0900,0.0913,0.1454,0.0956,0.1457,0.0866,0.0845,...,0.0574,0.0172,0.1475,0.2387,0.105400,0.023856,0.46450,0.405723,0.35669,0.410564
4,0.1249,0.0895,0.0903,0.0805,0.0742,0.1243,0.0430,0.1312,0.0884,0.0903,...,0.1545,0.0270,0.1633,0.1614,0.092582,0.025764,0.39389,0.311851,0.30023,0.315000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20395,0.0884,0.1095,0.0945,0.1328,0.1000,0.0863,0.0944,0.1219,0.0820,0.1005,...,0.1122,0.0164,0.0509,0.1049,0.101364,0.015347,0.14936,0.063340,0.04833,0.068961
20396,0.0655,0.0910,0.0916,0.1256,0.0987,0.0892,0.0776,0.1008,0.1037,0.1445,...,0.0201,-0.0496,-0.0169,0.0761,0.100727,0.022356,0.11479,0.066040,0.01597,0.059185
20397,0.0939,0.1008,0.0721,0.0903,0.0733,0.0742,0.0599,0.0913,0.0689,0.1034,...,0.0505,0.0256,0.0622,0.0983,0.083509,0.014318,0.14309,0.054281,0.06028,0.052993
20398,0.0923,0.0913,0.0992,0.1016,0.0820,0.0729,0.0728,0.0882,0.0576,0.0979,...,0.0383,-0.0061,0.0121,0.0938,0.086264,0.013601,0.12780,0.069816,0.04222,0.071676


Training and Evaluate XGBoost Model

In [9]:
# Encode target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['subject'])

def train_and_evaluate_model(X, y, feature_set_name):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = xgb.XGBClassifier(eval_metric='mlogloss')

    # Start timing for regular training
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()

    # Calculate elapsed time
    training_time = end_time - start_time

    # Evaluate model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    print(f"\nMetrics for {feature_set_name} (default parameter):")
    print(f" - Accuracy: {accuracy * 100:.2f}%")
    print(f" - Precision: {precision:.2f}")
    print(f" - Recall: {recall:.2f}")
    print(f" - F1-Score: {f1:.2f}")
    print(f"Training time: {training_time:.2f} seconds\n")

In [10]:
# Parameter GridSearch
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.2]
}

def train_with_gridsearch(X, y, param_grid, feature_set_name):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = xgb.XGBClassifier(eval_metric='mlogloss')

    # Set up GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=5,  # 5-fold cross-validation
        scoring='accuracy',
        verbose=1,
        n_jobs=-1
    )

    # Start timing for GridSearch
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    end_time = time.time()

    # Calculate elapsed time
    gridsearch_time = end_time - start_time

    # Get the best model and parameters
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"\nBest parameters for {feature_set_name} (with GridSearch): {best_params}")
    print(f"Best cross-validated accuracy: {best_score * 100:.2f}%")

    # Evaluate the best model on the test set
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    print(f"\nTest Metrics for {feature_set_name} (with GridSearch):")
    print(f" - Accuracy: {accuracy * 100:.2f}%")
    print(f" - Precision: {precision:.2f}")
    print(f" - Recall: {recall:.2f}")
    print(f" - F1-Score: {f1:.2f}")
    print(f"Training time (with GridSearch): {gridsearch_time:.2f} seconds\n")

Hasil

In [11]:
# Train and evaluate models for each feature set
train_and_evaluate_model(first_order_df, y, "First-Order Features")
train_and_evaluate_model(second_order_df, y, "Second-Order Features")
train_and_evaluate_model(combined_df, y, "Combined First and Second-Order Features")


Metrics for First-Order Features (default parameter):
 - Accuracy: 93.16%
 - Precision: 0.93
 - Recall: 0.93
 - F1-Score: 0.93
Training time: 14.00 seconds


Metrics for Second-Order Features (default parameter):
 - Accuracy: 44.71%
 - Precision: 0.44
 - Recall: 0.45
 - F1-Score: 0.44
Training time: 8.67 seconds


Metrics for Combined First and Second-Order Features (default parameter):
 - Accuracy: 93.21%
 - Precision: 0.93
 - Recall: 0.93
 - F1-Score: 0.93
Training time: 21.43 seconds



In [12]:
train_with_gridsearch(first_order_df, y, param_grid, "First-Order Features")
train_with_gridsearch(second_order_df, y, param_grid, "Second-Order Features")
train_with_gridsearch(combined_df, y, param_grid, "Combined First and Second-Order Features")

Fitting 5 folds for each of 8 candidates, totalling 40 fits



Best parameters for First-Order Features (with GridSearch): {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100}
Best cross-validated accuracy: 93.33%

Test Metrics for First-Order Features (with GridSearch):
 - Accuracy: 94.34%
 - Precision: 0.94
 - Recall: 0.94
 - F1-Score: 0.94
Training time (with GridSearch): 149.12 seconds

Fitting 5 folds for each of 8 candidates, totalling 40 fits

Best parameters for Second-Order Features (with GridSearch): {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Best cross-validated accuracy: 45.87%

Test Metrics for Second-Order Features (with GridSearch):
 - Accuracy: 46.74%
 - Precision: 0.46
 - Recall: 0.47
 - F1-Score: 0.45
Training time (with GridSearch): 64.18 seconds

Fitting 5 folds for each of 8 candidates, totalling 40 fits

Best parameters for Combined First and Second-Order Features (with GridSearch): {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100}
Best cross-validated accuracy: 93.38%

Test Metrics for Comb